In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchtext
import tqdm
from torchnlp.metrics import get_moses_multi_bleu
from torchtext.data import Field, BucketIterator
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

import tensorflow as tf
import tensorflow_datasets as tfds
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import linecache
import sys
import os
import re
import random
import time

In [2]:
!nvidia-smi

Thu Oct 31 14:15:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 430.40       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 44%   42C    P8     1W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Loading the dataset

In [3]:
try:
    os.mkdir("./datasets")
except FileExistsError:
    print("Directories already exists")

# getting descriptions
!wget https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.anno -O ./datasets/all.desc

# getting code
!wget https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.code -O ./datasets/all.code

Directories already exists
--2019-10-31 14:15:17--  https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.anno
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1382085 (1.3M) [text/plain]
Saving to: './datasets/all.desc'

./datasets/all.desc 100%[===================>]   1.32M  --.-KB/s    in 0.07s   

2019-10-31 14:15:18 (19.4 MB/s) - './datasets/all.desc' saved [1382085/1382085]

--2019-10-31 14:15:18--  https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.code
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 906732 (885K) [text/plain]
Saving to: './datasets/a

## Creating a token text encoder
An encoder will take a file and a splitting function and return an object able to encode and decode a string. It will also be able to save a vocab file and retrieve from file.

In [4]:
text = " append rel_to to string 'ForeignKey, (substitute the result for field_type.)"

# looks like code split need parenthesis to be matched in the same string, if not it gives an error...
def code_split(s):
    return [x.string for x in tokenize(BytesIO(s.encode('utf-8')).readline) if x.string != '' and x.string != "\n" and not x.string.isspace()][1:]

print(code_split(text))

['append', 'rel_to', 'to', 'string', "'", 'ForeignKey', ',', '(', 'substitute', 'the', 'result', 'for', 'field_type', '.', ')']


In [5]:
text = " append rel_to to string 'ForeignKey, (subs__titute the result' for field_type."

def string_split(s):
    return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(_|\W)', s))) # this will chunk all code properly by plits strings with quotes
#     return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(\\\'.*?\\\'|\\\".*?\\\"|_|\W)', s))) # this keeps the strings intact

print(string_split(text))

['append', 'rel', '_', 'to', 'to', 'string', "'", 'ForeignKey', ',', '(', 'subs', '_', '_', 'titute', 'the', 'result', "'", 'for', 'field', '_', 'type', '.']


## Making the input pipeline

In [6]:
def corpus_to_array(src_fp, tgt_fp):
    lines = []
    with open(src_fp, "r") as src_file, open(tgt_fp, "r") as tgt_file:
        for src, tgt in zip(src_file, tgt_file):
            lines.append((src, tgt))
    return lines

In [7]:
def filter_corpus(data, max_seq_length=200, tokenizer=string_split):
    return [(src, tgt) for src, tgt in data if len(string_split(src)) <= max_seq_length and len(string_split(tgt)) <= max_seq_length]

In [8]:
def samples_to_dataset(samples, src_field, tgt_field):
    """
    Args:
        samples: [(src_string),(tgt_string)]
        src/tgt_tokenizer: a func that takes a string and returns an array of strings
    """
    examples = []
    
    for sample in samples:
        src_string, tgt_string = sample
        examples.append(torchtext.data.Example.fromdict({"src":src_string, "tgt":tgt_string}, 
                                        fields={"src":("src",src_field), "tgt":("tgt",tgt_field)}))
        
    dataset = torchtext.data.Dataset(examples,fields={"src":src_field, "tgt":tgt_field})
    return dataset

In [9]:
data = corpus_to_array("datasets/all.desc", "datasets/all.code")
random.shuffle(data)

In [10]:
print("Max src length:", max([len(string_split(src)) for src, tgt in data]))
print("Max tgt length:", max([len(string_split(tgt)) for src, tgt in data]))

Max src length: 586
Max tgt length: 1087


In [11]:
print("Full dataset size:", len(data))
max_seq_length=200
data = filter_corpus(data, max_seq_length=200, tokenizer=string_split)
print("Limited dataset size:", len(data))

Full dataset size: 18805
Limited dataset size: 18781


In [12]:
SRC_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')
TGT_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')

dataset = samples_to_dataset(data, SRC_TEXT, TGT_TEXT)

train_dataset, val_dataset = dataset.split([0.9,0.1])

## Debugging dataset
This will be a small dataset of only 3 or 4 sentences to ensure the model can overfit.

In [13]:
# debug_data = [
#     ("my favourite foods are banana and toast","would you like banana and toast ?"),
#     ("my favourite foods are eggs and bacon and beans","would you like eggs and bacon and beans ?"),
#     ("my favourite food is chocolate","would you like chocolate ?"),
#     ("my favourite food is avocado","would you like avocado ?")
# ]

# other_data = [
#     ("what age is she ?", "she is 8 years old"),
#     ("what age is he ?", "he is 4 years old"),
#     ("how old are you ?", "i am 22 years old"),
#     ("how old am i ?", "you are 28 years old")
# ]

# SRC_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')
# TGT_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')

# train_dataset = val_dataset = samples_to_dataset(other_data, SRC_TEXT, TGT_TEXT)

# # train_dataset, val_dataset = dataset.split([0.7,0.3])

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0) # choose GPU from nvidia-smi 
print("Using:", device)

Using: cuda


In [15]:
SRC_TEXT.build_vocab(train_dataset)
TGT_TEXT.build_vocab(train_dataset)


sample = dataset[2].src
for tok, id in zip(sample, SRC_TEXT.numericalize([sample])):
    print("{} -> {}".format(tok, id.numpy()[0]))

use -> 72
_ -> 5
func -> 141
is -> 11
boolean -> 53
False -> 78
. -> 4


In [31]:
len(TGT_TEXT.vocab.itos)

4210

In [38]:
train_step(next(iter(train_iterator)))

NameError: name 'optimizer' is not defined

In [40]:
evaluate()

| EVALUATION |     0/   59 batches |
| EVALUATION |    50/   59 batches |
| EVALUATION | BLEU:  0.00 |


In [44]:
len(TGT_TEXT.vocab.itos)

4210

In [30]:
batch_size = 32

train_iterator = BucketIterator(
    train_dataset,
    batch_size = batch_size,
    repeat=True,
#     shuffle=True,
    sort_key = lambda x: len(x.src)+len(x.tgt),
    device = device)

valid_iterator = BucketIterator(val_dataset,
    batch_size = batch_size,
    sort_key = lambda x: len(x.src)+len(x.tgt),
    device = device)

# The iterator generates batches with padded length for sequences with similar sizes, a batch is [seq_length, batch_size]

for i, batch in enumerate(train_iterator):
    idx = 0
    print([SRC_TEXT.vocab.itos[id] for id in batch.src.cpu().numpy()[:,idx]])
    print(batch.src.cpu().numpy()[:,idx])
    print(batch.tgt.cpu().numpy()[:,idx])
    break

['<sos>', 'register', '.', 'tag', 'decorator', ',', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
[  2 256   4 179 128   6   3   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1]
[  2  95 205   5 137   3   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1]


## Building the model


Sample transformer without positional encoding, it uses the built in transformer model

In [18]:
rand_transformer_model = nn.Transformer() # uses default hyperparameters
src = torch.rand((10, 32, 512)) # [src_seq_length, batch_size, embedding_size]
tgt = torch.rand((20, 32, 512)) # [tgt_seq_length, batch_size, embedding_size]
rand_transformer_model(src, tgt).shape # [tgt_seq_length, batch_size, embedding_size]

torch.Size([20, 32, 512])

In [19]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [20]:
class TransformerModel(nn.Module):

    def __init__(self, src_vocab_size, tgt_vocab_size, embedding_size=512, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        
        self.embedding_size = embedding_size
        self.pos_encoder = PositionalEncoding(embedding_size, dropout)
        self.src_encoder = nn.Embedding(src_vocab_size, embedding_size)
        self.tgt_encoder = nn.Embedding(tgt_vocab_size, embedding_size)
        
        self.transformer = nn.Transformer(d_model=embedding_size, nhead=8, num_encoder_layers=4, num_decoder_layers=4, dim_feedforward=1024)
        self.decoder = nn.Linear(embedding_size, tgt_vocab_size)

        self.init_weights()
        self.tgt_mask = None

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz))).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.src_encoder.weight.data.uniform_(-initrange, initrange)
        self.tgt_encoder.weight.data.uniform_(-initrange, initrange)
        
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt):
        if self.tgt_mask is None or self.tgt_mask.size(0) != len(tgt):
            self.tgt_mask = self._generate_square_subsequent_mask(len(tgt)).to(device)

        src = self.src_encoder(src) * math.sqrt(self.embedding_size)
        src = self.pos_encoder(src)
        
        tgt = self.tgt_encoder(tgt) * math.sqrt(self.embedding_size)
        tgt = self.pos_encoder(tgt)
        
        output = self.transformer(src, tgt, tgt_mask=self.tgt_mask)
        output = self.decoder(output)
        return output

In [21]:
src_vocab_size = len(SRC_TEXT.vocab.stoi)
tgt_vocab_size = len(TGT_TEXT.vocab.stoi)

model = TransformerModel(src_vocab_size, tgt_vocab_size, dropout=0.2).to(device) 

In [22]:
model._generate_square_subsequent_mask(4)

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [23]:
mask = (torch.triu(torch.ones(4,4))).transpose(0,1)
print(mask)
print(mask.float())
print(mask.float().masked_fill(mask == 0, float('-inf')))
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
mask

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[1., -inf, -inf, -inf],
        [1., 1., -inf, -inf],
        [1., 1., 1., -inf],
        [1., 1., 1., 1.]])


tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [24]:
model.transformer.generate_square_subsequent_mask(4)

tensor([[0., 0., 0., 0.],
        [-inf, 0., 0., 0.],
        [-inf, -inf, 0., 0.],
        [-inf, -inf, -inf, 0.]])

In [29]:
src = torch.randint(0, src_vocab_size, (10,8), device=device) # [src_seq_length, batch_size]
tgt = torch.randint(0, tgt_vocab_size, (33,8), device=device) # [src_seq_length, batch_size]
model(src, tgt).shape 

torch.Size([33, 8, 4214])

In [30]:
criterion = nn.CrossEntropyLoss(ignore_index=TGT_TEXT.vocab.stoi['<pad>'])
lr = 0.005 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

tmp_out = []
tmp_trg = []

def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    tgt_vocab_size = len(TGT_TEXT.vocab.stoi)
    for i, batch in enumerate(train_iterator):
        encoder_input = batch.src
        decoder_input = batch.tgt[:-1]
        targets = batch.tgt[1:]
        
        optimizer.zero_grad()
        output = model(encoder_input, decoder_input)
#         print("src", batch.src)
#         print("decoder input", decoder_input)
#         print("prediced", output.argmax(dim=-1))
        tmp_out = output
        tmp_trg = targets
        loss = criterion(output.view(-1, tgt_vocab_size), targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        log_interval = 200
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_iterator), scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [59]:
best_val_loss = float("inf")
epochs = 20 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    print("######## Finished Epoch {} #########".format(epoch))
#     val_loss = evaluate(model, val_data)
#     print('-' * 89)
#     print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
#           'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
#                                      val_loss, math.exp(val_loss)))
#     print('-' * 89)

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         best_model = model

    scheduler.step()

|   200/10000 steps | lr 0.0050 | ms/batch 33.12 | loss  4.34 | ppl    76.89
|   400/10000 steps | lr 0.0050 | ms/batch 32.67 | loss  4.26 | ppl    71.01
|   600/10000 steps | lr 0.0047 | ms/batch 32.74 | loss  4.23 | ppl    69.03
|   800/10000 steps | lr 0.0047 | ms/batch 32.62 | loss  4.26 | ppl    70.90


KeyboardInterrupt: 

In [32]:
def greedy_decode_batch_ids(encoder_input, max_seq_length=50):
            batch_len = encoder_input.shape[1]
            sos_id = TGT_TEXT.vocab.stoi["<sos>"]
            decoder_input = torch.zeros((1, batch_len), dtype=torch.long, device=device).fill_(sos_id)
            
            for i in range(max_seq_length):
                output = model(encoder_input, decoder_input)
                last_pred = output[-1:].argmax(dim=2)

                decoder_input = torch.cat((decoder_input, last_pred))
            return decoder_input

In [33]:
def nltk_bleu(refrence, prediction):
    """
    Implementation from ReCode
    and moses multi belu script sets BLEU to 0.0 if len(toks) < 4
    """
    ngram_weights = [0.25] * min(4, len(refrence))
    return sentence_bleu([refrence], prediction, weights=ngram_weights, 
                          smoothing_function=SmoothingFunction().method3)

nltk_bleu(np.array([1,2,3,4,5,6]), np.array([1,2,5,6]))

0.2740311596835683

In [34]:
def evaluate():
    model.eval() # Turn on the evaluation mode
    total_loss = 0.
    with torch.no_grad():
        results = []
        targets = []
        BLEU_scores = []
        for i, batch in enumerate(valid_iterator):
            encoder_inputs = batch.src
            target = batch.tgt
            predictions = greedy_decode_batch_ids(encoder_inputs, max_seq_length=6)
            
            results += predictions.transpose(0,1).cpu().tolist()
            targets += target.transpose(0,1).cpu().tolist()
            if i % 50 == 0:
                print("| EVALUATION | {:5d}/{:5d} batches |".format(i, len(valid_iterator)))
        
        for r_ids, target in zip(results, targets):
            eos_id = TGT_TEXT.vocab.stoi["<eos>"]
            eos_index = r_ids.index(eos_id) if eos_id in r_ids else None
            cut_ids = r_ids[:eos_index]
            filtered_ids = [id for id in cut_ids if id not in [0,1,2,3]]
            filtered_target_ids = [id for id in target if id not in [0,1,2,3]]
            BLEU_scores.append(nltk_bleu(filtered_target_ids, filtered_ids))
        print("| EVALUATION | BLEU: {:5.2f} |".format(np.average(BLEU_scores)))
        

In [45]:
def train_step(batch):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    tgt_vocab_size = len(TGT_TEXT.vocab.itos)
    encoder_input = batch.src
    decoder_input = batch.tgt[:-1]
    targets = batch.tgt[1:]

    optimizer.zero_grad()
    output = model(encoder_input, decoder_input)

    loss = criterion(output.view(-1, tgt_vocab_size), targets.view(-1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    elapsed = time.time() - start_time
    return loss

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=TGT_TEXT.vocab.stoi['<pad>'])
lr = 0.009 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(steps=10000, log_interval=200, learning_interval=4000, eval_interval=1000):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    step = 1
    for batch in train_iterator:
        loss = train_step(batch)
        total_loss += loss.item()
        
        if step % log_interval == 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| {:5d}/{:5d} steps | '
                  'lr {:02.4f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    step, steps, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        
        if step % eval_interval == 0:
            print("Evaluating model")
            evaluate()
            model.train()
        
        if step % learning_interval == 0:
            scheduler.step()
        
        step += 1
        if step >= steps:
            print("Finished training")
            return

train(steps=500000,eval_interval=4000)

|   200/500000 steps | lr 0.0090 | ms/batch 43.37 | loss  6.58 | ppl   723.70
|   400/500000 steps | lr 0.0090 | ms/batch 46.31 | loss  5.67 | ppl   291.41
|   600/500000 steps | lr 0.0090 | ms/batch 46.63 | loss  5.46 | ppl   234.40
|   800/500000 steps | lr 0.0090 | ms/batch 45.74 | loss  5.30 | ppl   200.52
|  1000/500000 steps | lr 0.0090 | ms/batch 44.21 | loss  5.16 | ppl   173.62
|  1200/500000 steps | lr 0.0090 | ms/batch 46.22 | loss  5.03 | ppl   152.82
|  1400/500000 steps | lr 0.0090 | ms/batch 46.60 | loss  4.94 | ppl   139.73
|  1600/500000 steps | lr 0.0090 | ms/batch 45.94 | loss  4.87 | ppl   130.12
|  1800/500000 steps | lr 0.0090 | ms/batch 44.72 | loss  4.77 | ppl   117.71
|  2000/500000 steps | lr 0.0090 | ms/batch 46.12 | loss  4.68 | ppl   107.48
|  2200/500000 steps | lr 0.0090 | ms/batch 45.61 | loss  4.64 | ppl   103.67
|  2400/500000 steps | lr 0.0090 | ms/batch 45.27 | loss  4.56 | ppl    95.41
|  2600/500000 steps | lr 0.0090 | ms/batch 46.67 | loss  4.52 |

### Evaluating one sample

In [32]:
" ".join([SRC_TEXT.vocab.itos[i] for i in [ 2,21,83,13,10, 4, 5, 5,83, 4, 3]])

'<sos> substitute args for self . _ _ args . <eos>'

In [33]:
" ".join([TGT_TEXT.vocab.itos[i] for i in [ 2,12,5,4]])

'<sos> self . _'

In [34]:
def translate(s):
    src_ids = SRC_TEXT.numericalize([["<sos>"] + SRC_TEXT.preprocess(s) + ["<eos>"]], device=device)
#     src_ids = torch.tensor([ [2],[21],[83],[13],[10], [4], [5], [5],[83], [4], [3]], device=device)
    print("SRC ids shape:",src_ids)
    model.eval
    with torch.no_grad():
        sos_id = TGT_TEXT.vocab.stoi["<sos>"]
#         decoder_input = torch.zeros((1, 1), dtype=torch.long, device=device).fill_(sos_id)
        decoder_input = torch.tensor(np.array([ [2]]), device=device)
#         print("Decoder input shape:", decoder_input.shape)
        
        for i in range(10):
#             print("Decoder input", decoder_input)
            output = model(src_ids, decoder_input)
#             print(model.tgt_mask)
#             print("output shape:", output)
#             print("predicted ids:", output.argmax(dim=-1))
            last_pred = output[-1:].argmax(dim=2)
#             decoder_input[i+1][0] = last_pred
#             print("last pred:", TGT_TEXT.vocab.itos[last_pred.cpu().numpy()[0][0]], last_pred.cpu().numpy()[0][0])
            print(TGT_TEXT.vocab.itos[last_pred.cpu().numpy()[0][0]],'', end = '')
            
            decoder_input = torch.cat((decoder_input, last_pred))
#             print("Decoder input", decoder_input)
#             break

translate("for every key in keys,")

SRC ids shape: tensor([[  2],
        [ 13],
        [ 62],
        [ 35],
        [ 40],
        [255],
        [  6],
        [  3]], device='cuda:0')
for key in keys : <eos> [ key ] : 

In [36]:
nltk_bleu(np.array([torch.tensor([1.0]),torch.tensor([2.0])]), [1,2,5,6])

0.6389431042462724

In [37]:
np.array([torch.tensor([1.0]),torch.tensor([2.0])])

array([1., 2.], dtype=float32)

In [38]:
def evaluate(eval_model, valid_iterator):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    tgt_vocab_size = len(TGT_TEXT.vocab.stoi)
    with torch.no_grad():
        results = []
        for j, batch in enumerate(valid_iterator):
            encoder_input = batch.src
            sos_id = TGT_TEXT.vocab.stoi["<sos>"]
            decoder_input = torch.zeros((1, batch.tgt.shape[1]), dtype=torch.long, device=device).fill_(sos_id)
            targets = batch.tgt[1:]
            
#             for i in range(max_seq_length):
            for i in range(10):
                output = eval_model(encoder_input, decoder_input)
                last_pred = output[-1:].argmax(dim=2)
                # top k sampling
                # indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
                # F.softmax()
                decoder_input = torch.cat((decoder_input, last_pred))
            
            if ((j+1) % 40 == 0):
                print("Done {}/{}".format(j, len(valid_iterator)))
            
            for i in range(decoder_input.shape[1]):
                ref_ids = np.array([id for id in batch.tgt.transpose(0,1)[i] if id not in [0,1,2,3]])
                pred_ids = np.array([id for id in decoder_input.transpose(0,1)[i] if id not in [0,1,2,3]])
                src_ids = np.array([id for id in batch.src.transpose(0,1)[i] if id not in [0,1,2,3]])
                
                src_sent = " ".join([SRC_TEXT.vocab.itos[id] for id in src_ids])
                predicted_sent = " ".join([TGT_TEXT.vocab.itos[id] for id in pred_ids])
                ref_sent = " ".join([TGT_TEXT.vocab.itos[id] for id in ref_ids])
                                
                result = {
                            "ref_ids": ref_ids,
                            "pred_ids": pred_ids,
                            "BLEU": nltk_bleu(np.array(ref_ids), np.array(pred_ids)),
                    
                            "src_sent": src_sent, 
                            "ref_sent": ref_sent, 
                            "pred_sent": predicted_sent
                         }
                results.append(result)
        
        overall_BLEU = np.mean([r["BLEU"] for r in results])
        return results, overall_BLEU

Moses Multi-BLEU perl script returns 0.0 for any sentence less than 4 tokens long.
It will be best to use a function by NLTK

In [39]:
get_moses_multi_bleu(["this is a test"], ["this is a for"])

0.0

In [44]:
results, BLEU = evaluate(model, valid_iterator)

Done 39/118
Done 79/118


In [45]:
for r in results:
    ref_ids  = [id.data.tolist() for id in r["ref_ids"]]
    pred_ids  = [id.data.tolist() for id in r["pred_ids"]]
#     print(np.array(r["pred_ids"]))
    b = nltk_bleu(ref_ids, pred_ids)
    r["BLEU"] = b
    print("Query\t\t: {}\nTrue Translation: {}\nPrediction\t: {}\nBLEU\t\t:{}\n\n".format(r["src_sent"], r["ref_sent"], r["pred_sent"], b))

print("Overall BLEU:",np.average([r["BLEU"] for r in results]))

Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6389431042462724


Query		: do nothing .
True Translation: pass
Prediction	: pass
BLEU		:1.0


Query		: do nothing .
True Translation: pass
Prediction	: pass
BLEU		:1.0


Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6389431042462724


Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6389431042462724


Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6389431042462724


Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6389431042462724


Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6389431042462724


Query		: return nothing .
True Translation: return
Prediction	: return
BLEU		:1.0


Query		: do nothing .
True Translation: pass
Prediction	: pass
BLEU		:1.0


Query		: do nothing .
True Translation: pass
Prediction	: pass
BLEU		:1.0


Query		: try ,
True Translation: try :
Prediction	: try : : :
BLEU		:0.6